In [ ]:
###############################################################
################## N U M E R I C   E I G E N ##################
###############################################################

import numpy as np

'''
              A dot b_k
    b_k1 = --------------- 
           || A dot b_k ||
'''
def power_iteration(A, num_simulations):
    # b is random vector with size of A
    b_k = np.random.rand(A.shape[1])
    
    for _ in range(num_simulations):
        # Ab
        b_k1 = np.dot(A, b_k)

        # || b ||
        b_k1_norm = np.linalg.norm(b_k1)

        # b_k = Ab / ||b||
        b_k = b_k1 / b_k1_norm

    return b_k


'''
               b' dot A dot b
    R(A, b) = ----------------
                 b' dot b
'''
def rayleigh_quotient(A, b):
    return np.dot(b.transpose(), np.dot(A, b)) / np.dot(b.transpose(), b)

##############
# Application:
##############

# Always possible to get first eigenvalue and eigenvector:

# Resource: http://macs.citadel.edu/chenm/344.dir/08.dir/lect4_2.pdf (Chapter 3, Hotelling Deflation Method)
#
# With help of deflation you can calculate the remaining eigenvectors and -values.
# This is only possible if matrix can be diagonalized!
# If matrix can not be diagonalized, only the eigenvalues are correct. 
# Eigenvectors are nonsense in this case.

iterations = 100

A_22 = np.array([ [0.5, 0.5], 
                  [0.2, 0.8]])

A_33 = np.array([ [-4,  14, 0],
                  [-5,  13, 0],
                  [-1,   0, 2]])

A = A_22
B = A

# Main calculation loop
us = [] # iteration vectors
ls = [] # eigenvalues (lambdas)
xs = [] # deflation vectors
for i in range( B.shape[1] ):
    # get iteration vector
    us.append(power_iteration(B, iterations))
    # get eigen value
    ls.append(rayleigh_quotient(B, us[i]))
    
    # get deflation vector
    xs.append(
        B[i,:].transpose() / (ls[i] * us[i][i])
    )
    
    # get new A
    B = B - ls[i] * np.outer( us[i], xs[i].transpose() )
    
# Calculation of eigenvectors
vs = []
vs.append(us[0]) # we know that v0 = u0
for i in range(1, B.shape[1]):
    
    # v_i = (lambdas(i) - lambdas(i-1)) * iteration_vectors(i) + 
    #       lambdas(i-1) * (deflation_vectors(i-1)' * iteration_vectors(i) * eigenvectors(i-1)
    vs.append(
        (ls[i] - ls[0]) * us[i] + 
        ls[i-1] * np.matmul(xs[0].transpose(), us[i]) * vs[0]
    ) 

# Calculation of A based on eigenvalues and -vectors
# Defined by eigen value decomposition:
# A*X = X*L <=> A = X*L*X_i
# with L = Matrix containing eigenvalues as diagonal
# and  X = Matrix containing eigenvectors as columns
# sorting of values and vectors must be the same
L = np.zeros(A.shape)
X = np.zeros(A.shape)
for i in range(len(ls)):
    L[i, i] = ls[i]
    X[:,i] = vs[i]

A_r = np.matmul(X, np.matmul(L, np.linalg.inv(X)))

for i in range(len(ls)):
    print("\u03BB_" + str(i) + ": %0.3f" % ls[i])
    print("v_" + str(i) + ": " + str(vs[i]) + "\n")

np.set_printoptions(1, suppress = True)
print("Reconstructed Matrix A_r:\n", str(A_r),"\n")
print("A - A_r:\n", str(A-A_r))
